<a href="https://colab.research.google.com/github/ParkSongJi/BPE-WPT/blob/main/%EA%B3%BC%EC%A0%9C_BPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. BPE

In [ ]:
# 단어 dictionary 만들기
ori_dictionary = {
    'low': 5,
    'lower': 7,
    'newest': 6,
    'widest': 3
}
ori_dictionary


{'low': 5, 'lower': 7, 'newest': 6, 'widest': 3}

# 데이터 전처리
# -> 단어들을 글자(chracter) 단위로 분리
##### low    -> l o w
##### lower  -> l o w e r
##### newest -> n e w e s t
##### widest -> w i d e s t

In [ ]:
dictionary = {}

for words, count in ori_dictionary.items():
  new_word = ''
  for i in words:
    new_word += i + ' '
  dictionary[new_word + '-'] = count
dictionary

{'l o w -': 5, 'l o w e r -': 7, 'n e w e s t -': 6, 'w i d e s t -': 3}

# 1회차
### 분리된 글자를 쌍으로 조합시 횟수 확인

In [ ]:
import collections
import re

In [ ]:
# defaultdict(int): 초기값 0으로 설정
pairs = collections.defaultdict(int)

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)-1):
     pairs[symbols[i],symbols[i+1]] += count
print(pairs)

defaultdict(<class 'int'>, {('l', 'o'): 12, ('o', 'w'): 12, ('w', '-'): 5, ('w', 'e'): 13, ('e', 'r'): 7, ('r', '-'): 7, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '-'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3})


In [ ]:
# vocabulary 만들기
vocabulary = set()

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)):
    if symbols[i] == '-':
        pass
    else:
        vocabulary.add(symbols[i])

vocabulary

{'d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w'}

### 빈도수 높은 횟수를 조합 확인

In [ ]:
# 횟수 가 높은 세트 확인
best = max(pairs, key=pairs.get)
best

('w', 'e')

### 조합을 dictionary에 업데이트

In [ ]:
new_set = ' '.join(best)
new_set

'w e'

In [ ]:
new_set.replace(" ", "")

'we'

In [ ]:
new_dictionary = {}

for word, count in dictionary.items():
  if new_set in word:
    new_word = word.replace(new_set, new_set.replace(" ", ""))
    new_dictionary[new_word] = dictionary[word]
  else:
    new_dictionary[word] = count
new_dictionary

{'l o w -': 5, 'l o we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}

In [ ]:
# l o w e r -> w e -> l o we r
# 5

# 2회차
# 1회차랑 똑같이 진행

In [ ]:
dictionary = new_dictionary

pairs = collections.defaultdict(int)

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)-1):
     pairs[symbols[i],symbols[i+1]] += count
print(pairs)

defaultdict(<class 'int'>, {('l', 'o'): 12, ('o', 'w'): 5, ('w', '-'): 5, ('o', 'we'): 7, ('we', 'r'): 7, ('r', '-'): 7, ('n', 'e'): 6, ('e', 'we'): 6, ('we', 's'): 6, ('s', 't'): 9, ('t', '-'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3, ('e', 's'): 3})


In [ ]:
for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)):
    if symbols[i] == '-':
        pass
    else:
        vocabulary.add(symbols[i])

vocabulary

{'d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'we'}

In [ ]:
best = max(pairs, key=pairs.get)
best

('l', 'o')

In [ ]:
new_set = ' '.join(best)
new_set

'l o'

In [ ]:
new_dictionary = {}

for word, count in dictionary.items():
  if new_set in word:
    new_word = word.replace(new_set, new_set.replace(" ", ""))
    new_dictionary[new_word] = dictionary[word]
  else:
    new_dictionary[word] = count
new_dictionary

{'lo w -': 5, 'lo we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}

# 위 순서대로 원하는 만큼 돌리고 dictionary 및 vocabulary 상시 업데이트

# BPE 알고리즘


In [ ]:
import re
import collections

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def make_vocabulary(dictionary):
  global vocabulary
  for words, count in dictionary.items():
    symbols = words.split()
    for i in range(len(symbols)):
        if symbols[i] == '-':
            pass
        else:
            vocabulary.add(symbols[i])

  return vocabulary

def merge_dictionary(pair, dictionary):
    new_dictionary = {}

    for word, count in dictionary.items():
        if pair in word:
            new_word = word.replace(pair, pair.replace(" ", ""))
            new_dictionary[new_word] = dictionary[word]
        else:
            new_dictionary[word] = count
    return new_dictionary

In [ ]:
dictionary = {'l o w -': 5,
         'l o w e r -': 7,
         'n e w e s t -':6,
         'w i d e s t -':3
         }

vocabulary = set()
num_merges = 10

for i in range(num_merges):
    vocabulary = make_vocabulary(dictionary)
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    new_set = ' '.join(best)
    dictionary = merge_dictionary(new_set, dictionary)

    print("new merge: {}".format(best))
    print('dictionary: {}'.format(dictionary))
    print('vocabulary: {}'.format(vocabulary))
    print('=' * 300)

new merge: ('w', 'e')
dictionary: {'l o w -': 5, 'l o we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}
vocabulary: {'o', 'e', 'l', 'r', 'n', 'i', 't', 'd', 's', 'w'}
new merge: ('l', 'o')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}
vocabulary: {'o', 'e', 'l', 'r', 'we', 'n', 'i', 't', 'd', 's', 'w'}
new merge: ('s', 't')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we st -': 6, 'w i d e st -': 3}
vocabulary: {'o', 'lo', 'e', 'l', 'r', 'we', 'n', 'i', 't', 'd', 's', 'w'}
new merge: ('st', '-')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'o', 'lo', 'e', 'l', 'r', 'we', 'n', 'i', 't', 'st', 'd', 's', 'w'}
new merge: ('lo', 'we')
dictionary: {'lo w -': 5, 'lowe r -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'st-', 'o', 'lo', 'e', 'l', 'r', 'we', 'n', 'i', 't', 'st', 'd', 's', 'w'}
new merge: ('lowe', 'r')
dictionary: {'lo w -': 5, 'lower -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'s